In [1]:
import os
os.environ["CT2_USE_MKL"] = "0"
os.environ["CT2_VERBOSE"] = "1"

In [4]:
from faster_whisper import WhisperModel

model_size = "small.en"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cpu", cpu_threads=4)

[2023-04-18 06:49:23.029] [ctranslate2] [thread 2910] [info] Loaded model /home/chiiyeh/.cache/huggingface/hub/models--guillaumekln--faster-whisper-small.en/snapshots/e0e3c0a16c844a994ca4d6d1318ce35f68236052 on device cpu:0
[2023-04-18 06:49:23.029] [ctranslate2] [thread 2910] [info]  - Binary version: 6
[2023-04-18 06:49:23.029] [ctranslate2] [thread 2910] [info]  - Model specification revision: 3
[2023-04-18 06:49:23.029] [ctranslate2] [thread 2910] [info]  - Selected compute type: float32
[2023-04-18 06:49:23.029] [ctranslate2] [thread 2910] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


In [5]:
import time

start = time.time()
segments, info = model.transcribe("GovTech_MFTMEP1_16000_mono_16.wav", beam_size=5, language='en')

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

print(f'Time: {time.time() - start}')

Detected language 'en' with probability 1.000000
[0.00s -> 12.56s]  Welcome to My Friend Tell Me One. Today, we're going to speak to the Go Bros, two engineers
[12.56s -> 13.96s]  who work in DevOps.
[13.96s -> 19.84s]  Okay, hi, my name is Ryan. I work on my course feature. So basically, I'm doing DevOps.
[20.08s -> 20.96s]  Same as the guy.
[28.40s -> 34.64s]  The textbook answer is DevOps is a culture. So in other words, the whole team knows how to do
[34.64s -> 39.84s]  operations, some parts of operations as well as development, giving the responsibility of
[39.84s -> 42.40s]  what operators normally do to developers.
[42.40s -> 46.24s]  It's kind of a wide term. So depending on where you go, different organizations,
[46.24s -> 55.68s]  different teams have different definition of it. So yeah, so it's not a new thing. Actually,
[55.68s -> 58.56s]  it's been around since 1993, since the Agile Manifesto came about.
[62.32s -> 67.36s]  Like, the 10th learning is cool. School cannot t

In [6]:
os.environ['LD_LIBRARY_PATH']

KeyError: 'LD_LIBRARY_PATH'

In [1]:
%load_ext line_profiler

In [5]:
from stream import WhisperModel
import logging, sys

model_size = "medium.en"

# Run on GPU with FP16
# model = WhisperModel(model_size, compute_type="int8", device="cpu", cpu_threads=0)
model = WhisperModel(model_size, download_root='/home/chiiyeh/dev/ct_model')
consoleHandler = logging.StreamHandler(stream=sys.stdout)
consoleHandler.setLevel(logging.INFO)
model.logger.addHandler(consoleHandler)
model.logger.setLevel(logging.INFO)




[2023-10-06 07:34:20.236] [ctranslate2] [thread 38125] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.


In [13]:
!$PWD

/bin/bash: /home/chiiyeh/dev: Is a directory


In [6]:
tokenizer, options, stream_options = model.init_options(
    beam_size=1,
    best_of=1,
    condition_on_previous_text=True, 
    max_prompt_tokens=100,
    drop_dup_prompt=True,
    is_prompt_sentence=True,
    finalised_segment_gap=3,
    rm_seconds=0.3, 
    word_timestamps=True, 
    without_timestamps=True, 
    # log_prob_threshold=-2, 
    max_tokens_per_second=5,
    use_prefix=False,
    prefix_drop_num_tokens=4,
    drop_prefix_duration=0,
    )
# model.warm_start(tokenizer, options, stream_options)

In [5]:
model.warm_start(tokenizer, options, stream_options)

time offset: 0.0
segment duration: 5.0
time offset: 0.0
segment duration: 15.0
time offset: 0.0
segment duration: 8.0


In [6]:
%lprun -f model.simulate_streaming -f model.generate_segments model.simulate_streaming("GovTech_MFTMEP1_16000_mono_16.wav", tokenizer, options, stream_options, min_interval=3)

time offset: 0.0
segment duration: 3.0
Processing segment at 00:00.000
Compression Ratio: 0.3333333333333333
Temperature: 0
token length: 5
limit: 18
Text: 
Not Finalised: 0.0 2.92  🎵
Processing time: 4.265393501999824
time offset: 0.0
segment duration: 7.2700000000000005
Processing segment at 00:00.000
Compression Ratio: 0.5
Temperature: 0
token length: 4
limit: 44
Text: 
Not Finalised: 0.0 7.24  Welcome.
Processing time: 5.012761594999574
time offset: 0.0
segment duration: 12.280000000000001
Processing segment at 00:00.000
Compression Ratio: 1.0459770114942528
Temperature: 0
token length: 28
limit: 74
Text: 
0.0 9.6  Welcome to My Friend Tell Me One.
Not Finalised: 9.6 11.76  Today we're going to speak to the Go Bros.
Not Finalised: 11.76 12.26  Two engineers.
Processing time: 5.809826906000126
time offset: 9.6
segment duration: 8.49
Processing segment at 00:09.600
Compression Ratio: 1.1578947368421053
Temperature: 0
token length: 39
limit: 51
Text: 
9.6 13.76  Today we're going to s

Timer unit: 1e-09 s

Total time: 205.161 s
File: /home/chiiyeh/dev/stream.py
Function: simulate_streaming at line 287

Line #      Hits         Time  Per Hit   % Time  Line Contents
   287                                               def simulate_streaming(
   288                                                   self,
   289                                                   audio_file: str,
   290                                                   tokenizer: Tokenizer,
   291                                                   options: TranscriptionOptions,
   292                                                   stream_options: StreamOptions,
   293                                                   min_interval: float = 3,
   294                                               ):
   295         1       3409.0   3409.0      0.0          sampling_rate = self.feature_extractor.sampling_rate
   296         1   48143365.0 48143365.0      0.0          full_audio = decode_audio(audio_file, samp

In [5]:
tokenizer.no_timestamps

50362

In [6]:
tokenizer.timestamp_begin

50363

In [7]:
tokenizer.sot

50257

In [8]:
tokenizer.sot_prev

50360

In [15]:
tokenizer, options, stream_options = model.init_options(
    beam_size=1,
    best_of=3,
    condition_on_previous_text=True, 
    no_repeat_ngram_size=3,
    max_prompt_tokens=100,
    drop_dup_prompt=True,
    is_prompt_sentence=True,
    finalised_segment_gap=3,
    rm_seconds=0.3, 
    word_timestamps=True, 
    without_timestamps=True, 
    # log_prob_threshold=-2, 
    max_tokens_per_second=5,
    use_prefix=False,
    prefix_drop_num_tokens=4,
    drop_prefix_duration=0,
    initial_prompt="This is the start of the conversation."
    )
# model.max_length=448
# model.warm_start(tokenizer, options, stream_options)

In [16]:
%lprun -f model.simulate_fixed_interval_streaming -f model.generate_segments model.simulate_fixed_interval_streaming("GovTech_MFTMEP1_16000_mono_16.wav", tokenizer, options, stream_options, interval=7)

time offset: 0.0
segment duration: 7.0
time offset: 0.0
segment duration: 7.0
Not Finalised: log_prob,Fallback,outside_bound, 0.0 4.68  Music
Processing time: 4.228480525998748
time offset: 0.0
segment duration: 14.0
time offset: 0.0
segment duration: 14.0
 6.380000000000002 9.32  Welcome to My Friend Tell Me One.
Not Finalised: outside_bound, 9.66 13.66  Today we're going to speak to the GoBros, two engineers who work in DevOps.
Processing time: 22.79311388300266
time offset: 9.32
segment duration: 11.0
time offset: 9.32
segment duration: 11.0
 9.32 13.68  Today we're going to speak to the Go Bros, two engineers who work in DevOps.
Processing time: 5.973286278007436
time offset: 13.68
segment duration: 14.0
time offset: 13.68
segment duration: 14.0
 14.52 15.6  Hi, my name is Ryan.
 16.08 17.28  I work on My Current Future.
 18.1 19.44  So basically I'm doing DevOps.
Processing time: 8.856779660010943
time offset: 19.44
segment duration: 15.0
time offset: 19.44
segment duration: 15.0


Timer unit: 1e-09 s

Total time: 302.649 s
File: /home/chiiyeh/dev/stream.py
Function: simulate_fixed_interval_streaming at line 429

Line #      Hits         Time  Per Hit   % Time  Line Contents
   429                                               def simulate_fixed_interval_streaming(
   430                                                   self,
   431                                                   audio_file: str,
   432                                                   tokenizer: Tokenizer,
   433                                                   options: TranscriptionOptions,
   434                                                   stream_options: StreamOptions,
   435                                                   interval: float = 3,
   436                                               ):
   437         1       1866.0   1866.0      0.0          sampling_rate = self.feature_extractor.sampling_rate
   438         1  101618187.0 101618187.0      0.0          full_audio = de

In [7]:
round(1.4)

1

In [11]:
stream_options

StreamOptions(finalised_segment_gap=2, out_of_bound_gap=5)

In [13]:
tokenizer

In [13]:
import os
import soundfile as sf
import time
import numpy as np
import importlib
import jeff_live
importlib.reload(jeff_live)
from jeff_live import WhisperModel_live

model_size = 'small.en'
model_dir = "pretrained_models/faster_whisper/"
# audio_file = "data/wav/IMDA_test_60s.wav"
audio_file = "GovTech_MFTMEP1_16000_mono_16.wav"

# Load model before model files have been downloaded
# faster_model_live = WhisperModel_live(model_size, compute_type="int8", download_root=model_dir)
# Load model after model files have been downloaded
faster_model_live = WhisperModel_live(model_size, compute_type="int8")

def run_audio(audio_file, faster_model_live):
  wav, fs = sf.read(audio_file)
  wav_len = len(wav)
  
  RATE = 16000
  CHUNK_S = 7
  CHUNK = CHUNK_S * RATE
  # max_time = 440.0
  max_time = -1
  compression_ratio_threshold = 2.4
  
  if max_time > 0:
      wav_len = min(wav_len, int(max_time*RATE))
  chunk_starts = list(range(0, wav_len, CHUNK))
  final_text = ''
  seek = 0.0
  for chunk_start in chunk_starts[0:]:
      t1 = time.time()
      non_final_text = ''
      repeat_text = ''
      start_idx = min(chunk_start, int(seek*RATE))
      end_idx = min(wav_len, chunk_start + CHUNK)
  
      # Make sure audio chunk is less than 30s
      # Skip to most recent 30s if longer
      start_idx = max(start_idx, end_idx - 30*RATE)
      start_time = start_idx/RATE
      end_time = end_idx/RATE
  
      print(f'Transcribing {start_time:.2f}->{end_time:.2f}s...')
      wav_chunk = np.array(wav[start_idx:end_idx], dtype=np.float32)
      
      segments_live, info = faster_model_live.transcribe(wav_chunk, beam_size=1, temperature=0, vad_filter=False, word_timestamps=False)
      seek = start_time
      for seg in segments_live:
          # print(seg)
          print(f'SEG {start_time+seg.start:.2f}->{start_time+seg.end:.2f}, seek:{start_time+seg.seek_live:.2f}'
                  + f', final:{seg.final}, compression_ratio: {seg.compression_ratio:.2f}, words: {seg.text.lstrip()}')
  
          if seg.compression_ratio > compression_ratio_threshold:
              repeat_text += seg.text
  
          elif seg.final:
              final_text += seg.text
              seek = start_time + seg.seek_live
          else:
              non_final_text = seg.text
  
      t2 = time.time()
      time_elapsed = t2 - t1
      print(f'Output: {final_text.lstrip()} | {non_final_text.lstrip()}')
      if repeat_text:
          print(f'REPEAT: {repeat_text.lstrip()}')
      print(f'Time elapsed: {time_elapsed:0.2f}s\n')

In [14]:
%lprun -f run_audio run_audio(audio_file, faster_model_live)

Transcribing 0.00->7.00s...
Processing audio with duration 00:07.000
Processing audio with duration 00:07.000
SEG 0.00->6.94, seek:6.94, final:False, compression_ratio: 0.38, words: Music
Output:  | Music
Time elapsed: 5.67s

Transcribing 0.00->14.00s...
Processing audio with duration 00:14.000
Processing audio with duration 00:14.000
SEG 0.00->9.60, seek:9.60, final:True, compression_ratio: 1.09, words: Welcome to My Friend Tell Me One.
SEG 9.60->14.00, seek:9.60, final:False, compression_ratio: 1.09, words: Today we're going to speak to the Go Bros, two engineers who work in DevOps.
Output: Welcome to My Friend Tell Me One. | Today we're going to speak to the Go Bros, two engineers who work in DevOps.
Time elapsed: 5.79s

Transcribing 9.60->21.00s...
Processing audio with duration 00:11.400
Processing audio with duration 00:11.400
SEG 9.60->13.76, seek:19.60, final:True, compression_ratio: 1.26, words: Today we're going to speak to the Go Bros, two engineers who work in DevOps.
SEG 1

Timer unit: 1e-09 s

Total time: 181.643 s
File: /tmp/ipykernel_10373/3897158399.py
Function: run_audio at line 20

Line #      Hits         Time  Per Hit   % Time  Line Contents
    20                                           def run_audio(audio_file, faster_model_live):
    21         1   13333197.0 13333197.0      0.0    wav, fs = sf.read(audio_file)
    22         1       1353.0   1353.0      0.0    wav_len = len(wav)
    23                                             
    24         1        337.0    337.0      0.0    RATE = 16000
    25         1        376.0    376.0      0.0    CHUNK_S = 7
    26         1        939.0    939.0      0.0    CHUNK = CHUNK_S * RATE
    27                                             # max_time = 440.0
    28         1        303.0    303.0      0.0    max_time = -1
    29         1        156.0    156.0      0.0    compression_ratio_threshold = 2.4
    30                                             
    31         1        406.0    406.0      0.0 